In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import BaselineOnly, accuracy
from surprise.accuracy import rmse
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV

In [7]:
filepath = 'C:/Users/samir/Documents/code/r/ads/proj4/'

def svd_knn(method = 11, n_epochs = [5, 10], reg_u = [13, 15, 17], reg_i = [8, 10, 12], cv = 5):
    reader = Reader()
    if method == 11:
        cols = ['movieId', 'userId', 'rating']
        ratings = pd.read_csv(filepath + 'data/ml-latest-small/ratings.csv', usecols = cols)
        data = Dataset.load_from_df(ratings, reader)
        train, test = train_test_split(data, test_size = 0.2)
        hyperparameters = {'k': [1], # using one nearest neighbor for cosine similarity
                           'n_epochs':n_epochs,
                           'bsl_options': {'method': ['als'],
                                           'reg_u': reg_u,
                                           'reg_i': reg_i},
                           'sim_options': {'name': ['cosine'],
                                           'user_based': [False]}}
        grid_search = GridSearchCV(KNNBasic, hyperparameters, measures=['rmse'], cv = cv)
        grid_search.fit(data)
        cv_rmse = grid_search.best_score['rmse']
        final_model = grid_search.best_estimator['rmse']
        final_model.fit(data.build_full_trainset())
        grid_search_results = pd.DataFrame.from_dict(grid_search.cv_results)
        predictions = final_model.test(test)
        rmse_pred = rmse(predictions, verbose = True)
        return final_model, grid_search_results, rmse_pred
    
svd_knn()    